# SQL

## Conexión a PostgreSQL usando SQLAlchemy

In [1]:
import os
import pandas as pd
from typing import Union
from sqlalchemy import create_engine, text

# Reemplaza USER y PASSWORD del .env
engine = create_engine("sqlite:///data/sql-murder-mystery.db")

# Probar la conexión
try:
    connection = engine.connect()
    print("Conexión exitosa")
    connection.close()
except Exception as e:
    print(f"Error al conectar: {e}")


def ejecutar_query(query: str) -> Union[pd.DataFrame, str]:
    """
    Ejecuta una consulta SQL en la base de datos y devuelve un DataFrame.

    Parámetros:
    - query (str): La consulta SQL a ejecutar.

    Retorna:
    - Union[pd.DataFrame, pl.DataFrame, str]: Retorna un DataFrame en pandas o polars dependiendo del parámetro 'devolver_polars',
      o un mensaje de éxito/error si la operación no es un SELECT.
    """
    
    with engine.connect() as connection:
        try:
            result = connection.execute(text(query))
            
            # Determinar si es una consulta SELECT para devolver datos
            if result.returns_rows:
                rows = result.fetchall()
                columns = result.keys()

                # Convertir filas en un diccionario de listas para crear correctamente el DataFrame en polars
                data_dict = {col: [row[idx] for row in rows] for idx, col in enumerate(columns)}

                # Verificación de coincidencia de columnas y filas
                return pd.DataFrame(data_dict)
            else:
                connection.commit()
                return f"Query ejecutada correctamente: {result.rowcount} filas afectadas."
        except Exception as e:
            return f"Error al ejecutar la query: {e}"


Conexión exitosa


In [10]:
pd.set_option('display.max_rows', 20)  # Mostrar todas las filas
pd.set_option('display.max_colwidth', None)  # Mostrar todas las columnas

In [17]:
query= """SELECT * 
FROM crime_scene_report
WHERE type = 'murder' 
AND city = 'SQL City'
AND date = '20180115'
"""
ejecutar_query(query=query)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [20]:
query = """SELECT * FROM person 
where name LIKE '%Annabel%' and address_street_name = 'Franklin Ave' """

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [31]:
query = """SELECT * FROM person 
where address_street_name = 'Northwestern Dr' 
ORDER BY address_number DESC """

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
2,53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147
3,73368,Torie Thalmann,773862,3697,Northwestern Dr,341559436
4,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829
...,...,...,...,...,...,...
45,12711,Norman Apolito,667757,599,Northwestern Dr,778264744
46,85280,Donn Duin,569122,540,Northwestern Dr,613873810
47,40336,Fredia Billeter,251082,404,Northwestern Dr,108056488
48,15171,Weldon Penso,336999,311,Northwestern Dr,131379495


In [32]:
query= 'SELECT * FROM interview WHERE person_id = 16371 or person_id = 14887 '

ejecutar_query(query=query)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [35]:
query= """SELECT * FROM get_fit_now_check_in
WHERE membership_id LIKE '%48Z%' and check_in_date = 20180109
"""

ejecutar_query(query=query)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [37]:
query= """SELECT * FROM get_fit_now_member
where id = '48Z7A' or id = '48Z55' """

ejecutar_query(query=query)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold
1,48Z7A,28819,Joe Germuska,20160305,gold


In [44]:
query= """SELECT * FROM drivers_license
WHERE plate_number LIKE '%H42W%'
and gender = 'male'

"""

ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [45]:
query= """ SELECT * FROM person 
where license_id = '423327' or license_id = '664760' """

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [48]:
query= """ SELECT * FROM get_fit_now_member
where person_id = '67318' or person_id = '51739' """

ejecutar_query(query=query)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


Como podemos observar, anteriormente annabel y morty nos han dado pistas con la fecha y placa del coche del asesino, eso nos ha llevado a 2 sospechosos a jeremy bower y a Joe Germuska. luego con la pista del gym pudimos corroborar que era jeremy bowers el que estuvo en la fecha y con el coche del asesino.